We import `netgen.gui` to open the netgen GUI for displaying the ferromagnet. 
We then define the geometry, and draw the box. 
The refinement of the mesh is controlled by `H_MAX` in `ngmesh`.

In [1]:
from netgen.csg import *
from ngsolve import *
from ngsolve.utils import grad, Grad  # If I don't import these explicitly, VSCode reads them as missing.
import netgen.gui  # this opens up the netgen ui
import Magnetisation_Functions as magfunc
import Elastic_Functions as elfunc
import numpy as np
import scipy.sparse as sp
T_MAX = 10.0  # The maximum time for the simulation
ALPHA = 1  # Dissipative constant in the LLG equation.
THETA = 0.75 # Should be strictly above 1/2 for unconditional stability
K = 0.1  # TIME STEP
KAPPA = 1  # Determines the relative strength of the elastic vs. magnetic parts.
H_MAX = 1  # Determines how fine the mesh should be.

In [2]:
def MakeGeometry():  # this makes a box, with labelled faces
    geometry = CSGeometry()
    left  = Plane(Pnt(0,0,0), Vec(-1,0,0)).bc("left")
    right = Plane(Pnt(1,1,1), Vec( 1,0,0)).bc("right")
    front = Plane(Pnt(0,0,0), Vec(0,-1,0)).bc("front")
    back  = Plane(Pnt(1,1,1), Vec(0, 1,0)).bc("back")
    bot   = Plane(Pnt(0,0,0), Vec(0,0,-1)).bc("bot")
    top   = Plane(Pnt(1,1,1), Vec(0,0, 1)).bc("top")

    cube = left * right * front * back * bot * top
    geometry.Add (cube)
    #cube = OrthoBrick(Pnt(0,0,0), Pnt(1,1,1))
    geometry.Add(cube)
    return geometry



ngmesh = MakeGeometry().GenerateMesh(maxh=H_MAX)
# ngmesh.Save("cube.vol")
mesh = Mesh(ngmesh)
Draw(mesh)

In [3]:
  # this helps me figure out which sides are which.
#fes_SCALAR = H1(mesh, order=1)
#myScalars = {"left":10, "right":10, "front":0, "back":0, "bot":5, "top":5}
#SCALAR_gfu = CoefficientFunction([myScalars[val] for val in mesh.GetBoundaries()])
#Draw(SCALAR_gfu, mesh, "SCALAR_gfu")


In [4]:
fes_mag = VectorH1(mesh, order=1)  # the finite element space for the magnetisation m_h^i
fes_disp = VectorH1(mesh, order=1, dirichlet="bot")  # the finite element space for the displacement u_h^i

print(f"mag_ndof={fes_mag.ndof}, disp_ndof={fes_disp.ndof},\n, dispfree_ndof={fes_disp.FreeDofs()}")
mag_gfu = GridFunction(fes_mag)
disp_gfu = GridFunction(fes_disp)
# body force and traction force

#f_body = CoefficientFunction((0,0,-1))
#g_surface = CoefficientFunction([-1 if bc=="left" else 0 for bc in mesh.GetBoundaries()])

mag_ndof=27, disp_ndof=27,
, dispfree_ndof=0: 010011011010011011010011011


In [5]:
# Randomly give initial magnetisation
mag_gfu = magfunc.give_random_magnetisation(mag_gfu)


# Test functions
v = fes_mag.TrialFunction()
phi = fes_mag.TestFunction()
# Building the linear system for the magnetisation

a_mag = BilinearForm(fes_mag)
a_mag += ALPHA*InnerProduct(v, phi)*dx  # α<v,Φ>
a_mag += THETA*K*InnerProduct(Grad(v), Grad(phi))*dx  # θk<∇v,∇Φ>
a_mag += InnerProduct(Cross(mag_gfu,v), phi)*dx  # <m×v,Φ>
a_mag.Assemble()

f_mag = LinearForm(fes_mag)
f_mag += -InnerProduct(grad(mag_gfu), grad(phi))*dx # -<∇m,∇Φ>
#f_mag +=  # magnetostrictive contribution.
f_mag.Assemble()

In [ ]:
# Test functions
u = fes_disp.TrialFunction()
psi = fes_disp.TestFunction()
# Building the linear system for the displacement
a_disp = BilinearForm(fes_disp)
a_disp += 1/(K*K)*InnerProduct(u, psi)*dx
a_disp += InnerProduct(elfunc.stress(elfunc.strain(u)), elfunc.strain(psi))*dx

f_disp = LinearForm(fes_disp)
#f_mag += InnerProduct(elfunc.stress(elfunc.strain_m()), ψ)*dx  # <Cε_m(Π m),ψ>
#f_mag += 1/K*InnerProduct(diff_t u, psi)*dx  # <d_t u^i, ψ>
f_disp += 1/(K*K)*InnerProduct(disp_gfu, psi)*dx  # <u^i, ψ>
#f_mag += InnerProduct(f_body, psi)*dx  # <f, ψ>
#f_mag += InnerProduct(g_surface, psi)*ds  # _/‾ g·ψ ds

In [10]:
B_T = magfunc.build_tangent_plane_matrix_transpose(mag_gfu)
with np.printoptions(threshold=np.inf):
    print(magfunc.give_magnetisation_update(a_mag, B_T, f_mag))

[-2.49587527e+00 -3.05448324e-01  1.29669631e+00 -4.38687568e+00
 -2.91616704e+00  7.63712428e-01 -6.64294968e-01 -2.11827253e+00
 -1.37912060e+00 -2.20972355e-03 -9.69699152e-01 -5.00706442e-01
  7.16243669e+00  1.12106729e+01 -8.51841861e-01 -2.38126475e+00
  9.82977321e-01 -3.82278777e+00 -1.30233609e+00 -4.49137600e+00
  1.71444798e+00 -3.33475940e+00  3.61487405e+00  4.01124053e+00
 -4.28308942e+00  1.26630049e+00  5.03985007e-02]


In [8]:
#print(mag_gfu.vec)
Draw(mag_gfu)
#print(magfunc.build_tangent_plane_matrix_transpose(mag_gfu))
